# Libraries


In [59]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from tkinter import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Reading DF - Train,Test,Split

In [25]:
df = pd.read_csv('liver.csv')
Xd = df.drop('selector',axis=1)
yd = df['selector']
X_train, X_test, y_train, y_test = train_test_split(Xd, yd, test_size=0.30)

# Methods

In [54]:
def draw_graph(event):
    fig = Figure(figsize=(7,6), dpi=100)
    ax = fig.add_subplot(111)
    sns.countplot(x='selector',data=df,ax=ax)
    canvas = FigureCanvasTkAgg(fig, master=graphSection)
    canvas.get_tk_widget().grid(row=3, column=0,padx=10,pady=10)


In [60]:
def applyLogistic(mcv,alkphos,sgpt,sgot,gammagt,drinks):
    logmodel = LogisticRegression()
    logmodel.fit(X_train,y_train)
    
    #apply prediction for input fields
    pred = logmodel.predict(np.array([mcv,alkphos,sgpt,sgot,gammagt,drinks]).reshape(1,-1))
    
    #apply predictions for data set
    predictions = logmodel.predict(X_test)
    
    #update class entry field
    classEntry.delete(0,"end")
    if(pred[0] == 1):
        classEntry.insert(0,"ABSENT")
    else:
        classEntry.insert(0,"PRESENT")
        
    #update accuracy field
    cr = classification_report(y_test,predictions,output_dict=True)
    accuracyEntry.insert(0,cr['1']['precision'])
    
    #update confusion matrix
    cf = confusion_matrix(y_test,predictions)
    canvas_id = confMatCanvas.create_text(10, 10, anchor="nw")
    confMatCanvas.itemconfig(canvas_id, text=cf)
    confMatCanvas.insert(canvas_id, 12, "")

In [61]:
def applyDT(mcv,alkphos,sgpt,sgot,gammagt,drinks):
    #create model
    rfc = RandomForestClassifier(n_estimators=110)
    rfc.fit(X_train, y_train)
    
    #apply prediction for input fields
    pred = rfc.predict(np.array([mcv,alkphos,sgpt,sgot,gammagt,drinks]).reshape(1,-1))
    
    #apply predictions for data set
    predictions = rfc.predict(X_test)
    
    #update class entry field
    classEntry.delete(0,"end")
    if(pred[0] == 1):
        classEntry.insert(0,"ABSENT")
    else:
        classEntry.insert(0,"PRESENT")
        
    #update accuracy field
    cr = classification_report(y_test,predictions,output_dict=True)
    accuracyEntry.insert(0,cr['1']['precision'])
    
    #update confusion matrix
    cf = confusion_matrix(y_test,predictions)
    canvas_id = confMatCanvas.create_text(10, 10, anchor="nw")
    confMatCanvas.itemconfig(canvas_id, text=cf)
    confMatCanvas.insert(canvas_id, 12, "")

In [62]:
#main driver function
def get_prediction(event):
    #take inputs
    mcv = int(mcvEntry.get())
    alkphos = int(alkphosEntry.get())
    sgpt = int(sgptEntry.get())
    sgot = int(sgotEntry.get())
    gammagt = int(gammagtEntry.get())
    drinks = float(drinksEntry.get())
    
    #check which algorithm user has selected 
    if(var.get()=="1"):
        applyLogistic(mcv,alkphos,sgpt,sgot,gammagt,drinks)
    else:
        applyDT(mcv,alkphos,sgpt,sgot,gammagt,drinks)
        

# GUI

To use place(), give the frame a width and height.
Frames on the same level should ude the same geometry manager

In [63]:
#main tkinter object called root
root = Tk()

#INPUT AREA
inputSection = LabelFrame(root,text="Input Fields")

mvcLabel =  Label(inputSection,text="mcv").grid(row=1,column=0)
mcvEntry = Entry(inputSection,width=10)
mcvEntry.grid(row=1,column=2)

alkphosLabel =  Label(inputSection,text="alkphos").grid(row=2,column=0)
alkphosEntry = Entry(inputSection,width=10)
alkphosEntry.grid(row=2,column=2)

sgptLabel = Label(inputSection,text="sgpt").grid(row=3,column=0)
sgptEntry = Entry(inputSection,width=10)
sgptEntry.grid(row=3,column=2)

sgotLabel = Label(inputSection,text="sgot").grid(row=4,column=0)
sgotEntry = Entry(inputSection,width=10)
sgotEntry.grid(row=4,column=2)

gammagtLabel = Label(inputSection,text="gammagt").grid(row=5,column=0)
gammagtEntry = Entry(inputSection,width=10)
gammagtEntry.grid(row=5,column=2)

drinksLabel = Label(inputSection,text="drinks").grid(row=6,column=0)
drinksEntry = Entry(inputSection,width=10)
drinksEntry.grid(row=6,column=2)

Label(inputSection,text="Test Size").grid(row=7,column=0)
slider = Scale(inputSection, from_=0, to=50, orient=HORIZONTAL)
slider.grid(row=7,column=2)

inputSection.place(x=5,y=5)

#ALGORITHM SELECTION AREA
algorithmSection = LabelFrame(root,text="Algorithms")

#var is a variable to get the radiobutton that has been selected
var = StringVar()
LogisticButton = Radiobutton(algorithmSection,text="Logistic",value="1",variable=var)
LogisticButton.grid(row=0,column=0)

DTButton = Radiobutton(algorithmSection,text="Decision Tree",value="2",variable=var)
DTButton.grid(row=1,column=0)

algorithmSection.place(x=5,y=200)

#PREDICT BUTTON
predict = Button(root,text="PREDICT")
predict.bind('<Button-1>',get_prediction)
predict.place(x=5,y=300)


#OUTPUT AREA
outputSection = LabelFrame(root,text="Prediction and Outputs")

accuracyLabel = Label(outputSection,text="Accuracy").grid(row=0,column=0)
accuracyEntry = Entry(outputSection,width=10)
accuracyEntry.grid(row=0,column=2)

confMatLabel = Label(outputSection,text="Confusion Matrix").grid(row=1,column=0)
confMatCanvas = Canvas(outputSection,width=90, height=90,bg="white")
confMatCanvas.grid(row=1,column=2,padx=5,pady=5)

classLabel = Label(outputSection,text="Predicted Class").grid(row=2,column=0)
classEntry = Entry(outputSection,width=10)
classEntry.grid(row=2,column=2,padx=5,pady=5)

outputSection.place(x=5,y=400)


#VISUALISATION AND PLOTTING AREA
graphSection = LabelFrame(root,text="Graphs/Plots")

plot = Button(graphSection,text="PLOT")
plot.bind('<Button-1>',draw_graph)
plot.grid(row=1,column=0,padx=10,pady=10)

graphSection.place(x=500,y=5)

root.mainloop()

C:\Users\Calvin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
